In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
import StringIO
import matplotlib
import matplotlib.pyplot as plt
import subprocess
import os
%matplotlib inline

In [75]:
df = pd.read_csv('debate.csv')
df.head(n=10)

,Line,Speaker,Text,Date
0,1,Holt,Good evening from Hofstra University in Hempst...,9/26/16
1,2,Audience,(APPLAUSE),9/26/16
2,3,Clinton,"How are you, Donald?",9/26/16
3,4,Audience,(APPLAUSE),9/26/16
4,5,Holt,Good luck to you.,9/26/16


In [268]:
from sklearn.cross_validation import train_test_split as sk_split
train = df[:1000]
test = df[1000:]
train.head(n=5)

,Line,Speaker,Text,Date
0,1,Holt,Good evening from Hofstra University in Hempst...,9/26/16
1,2,Audience,(APPLAUSE),9/26/16
2,3,Clinton,"How are you, Donald?",9/26/16
3,4,Audience,(APPLAUSE),9/26/16
4,5,Holt,Good luck to you.,9/26/16


In [269]:
x_text = []
y_cand = []
for i in range(len(train['Speaker'].values)):
    if train['Speaker'].values[i] == 'Trump':
        x_text.append(train['Text'].values[i])
        y_cand.append(0)
    elif train['Speaker'].values[i] == 'Clinton':
        x_text.append(train['Text'].values[i])
        y_cand.append(1)    

In [270]:
# build dictionary of all the words
word_to_index = {}
n = len(x_text)
print n
num = 0
for i in range(n):
    words = x_text[i].split()
    for word in words:
        if not word_to_index.has_key(word):
            word_to_index[word] = num
            num += 1

375


In [271]:
# count the words
total = [0] * 2
for i in range(n):
    total[y_cand[i]] += 1
    
counts = []
for label in range(2):
    counts.append([0] * num)
    for i in range(n):
        if y_cand[i] == label:
            words = x_text[i].split()
            for word in words:
                counts[label][word_to_index[word]] += 1

In [272]:
for key in word_to_index:
    c0 = counts[0][word_to_index[key]]
    c1 = counts[1][word_to_index[key]]
    if abs(c0 - c1) < (c0 + c1) / 4:
        continue
    if counts[0][word_to_index[key]] + counts[1][word_to_index[key]] > 50:
        print "key=", key, " counts=", counts[0][word_to_index[key]], counts[1][word_to_index[key]]

key= would  counts= 36 59
key= me  counts= 35 16
key= them  counts= 37 20
key= they  counts= 104 42
key= country  counts= 42 19
key= been  counts= 69 24
key= It's  counts= 45 14
key= can  counts= 19 51
key= so  counts= 59 35
key= say  counts= 40 11
key= We  counts= 67 40
key= going  counts= 98 43
key= where  counts= 37 17
key= many  counts= 37 15
key= But  counts= 99 46
key= or  counts= 36 15
key= at  counts= 93 49
key= you  counts= 263 88
key= all  counts= 62 29
key= very  counts= 94 37
key= I'm  counts= 55 20
key= --  counts= 173 51
key= that's  counts= 33 20
key= don't  counts= 53 17
key= it.  counts= 60 8
key= look  counts= 58 10
key= we're  counts= 34 18
key= have  counts= 236 141
key= who  counts= 21 49
key= she  counts= 98 4
key= it's  counts= 76 31
key= You  counts= 64 21
key= know  counts= 40 18
key= like  counts= 47 15
key= he  counts= 15 95


In [273]:
# calculate probability p[label][word] = log(p(word | label))

import sys
import math
def mylog(x):
    if x == 0:
        return sys.float_info.min
    else:
        return math.log(x)

def getProb(cnt):
    tot = sum(cnt)
    return [float(x) / tot for x in cnt]
def calculateTable(alpha = 1):
    global p
    p = []
    for label in range(2):
        p.append([0] * num)
        arr = getProb([x + alpha for x in counts[label]])
        p[label] = [mylog(x) for x in arr]
calculateTable()

In [274]:
def predictLabel(text):
    words = text.split()
    ans = [0] * 2
    for label in range(2):
        ans[label] = mylog(float(total[label]) / sum(total))
        for word in words:
            if not word_to_index.has_key(word):
                continue  # ignore for now
            else:
                ans[label] += p[label][word_to_index[word]]
    if ans[0] > ans[1]:
        return 0
    else:
        return 1

In [275]:
def testData(test_data):
    num_total = len(test_data)
    num_correct = 0
    for test_entry in test_data:
        num_correct += (predictLabel(test_entry[1]) == test_entry[0])
    return "%d out of %d: correct %.2lf" % (num_correct, num_total, float(num_correct) / num_total)


In [276]:
def getTestDataFromDataFrame(test):
    return_data = []
    for i in range(len(test['Speaker'].values)):
        if test['Speaker'].values[i] == 'Trump':
            return_data.append((0, test['Text'].values[i]))
        elif test['Speaker'].values[i] == 'Clinton':
            return_data.append((1, test['Text'].values[i]))
    return return_data
test_data = getTestDataFromDataFrame(test)

In [277]:
print test_data[0]


(1, 'We have to save as many lives as we possibly can.')


In [278]:
testData(test_data)


'194 out of 223: correct 0.87'

In [279]:
testData(zip(y_cand, x_text))
testData([(0,str1)])


'0 out of 1: correct 0.00'